In [1]:
#Ava Mehri, UIC, amehri2@uic.edu, 2021
#Cost Constrainted Shortest Paths Problem
#Determines the fastest way to get to a destination within a certain budget in a graph

In [2]:
import sys
from queue import PriorityQueue
from queue import LifoQueue

In [3]:
#in_file =  sys.argv[0]
in_file = 'g2.g'
#open the input file and store it
file = open(in_file, 'r').read()
#read the file line by line and store lines in a list
for line in file:
  lines = file.split('\n')
if (lines[-1] == ''):
    lines = lines[0:-1]
#remove spaces before each line
for i in range(len(lines)):
  lines[i] = lines[i].lstrip()
  lines[i] = lines[i].split(' ')
#transform strings to integers
for i in range(len(lines)):
  for j in range(len(lines[i])):
    lines[i][j] = int(lines[i][j])

In [4]:
#start =  sys.argv[1]
#destination =  sys.argv[2]
#budget =  sys.argv[0][3]
start = 0
destination = 6
budget = 12
graph = {}
P_vertices = {}
predecessor = {}
for i in range(len(lines)):
  u = lines[i][0]
  v = lines[i][1]
  c = lines[i][2]
  t = lines[i][3]
  #print( "graph:", graph)
  #print( "P_vertices", P_vertices)
  if not u in P_vertices:
    P_vertices.update({u:[]})
    predecessor.update({u:PriorityQueue()})
  if not v in P_vertices:
    P_vertices.update({v:[]})
    predecessor.update({v:PriorityQueue()})

  if not u in graph:
    graph.update({u:{v: (c,t)}})
  if u in graph:
    if not v in graph[u]:
      graph[u].update({v: (c,t)})

predecessor[start].put((0,0,start))

In [5]:
Q = PriorityQueue()
Q.put((0,0,start))
while not Q.empty():
  temp = Q.get()
  cost = temp[0]
  time = temp[1]
  u = temp[2]
  if not P_vertices[u]:
    P_vertices[u].append((cost, time, predecessor[u].get()[2]))
  else:
    if cost >= P_vertices[u][-1][0] and time < P_vertices[u][-1][1]:
      P_vertices[u].append((cost, time, predecessor[u].get()[2]))

  V_out = list(graph.keys())
  if u in V_out:
    g = graph[u]
    neighbors = list(g.keys())
    for v in neighbors:
      cost_v = cost + g[v][0]
      time_v = time + g[v][1]
      if not P_vertices[v]:
        Q.put((cost_v, time_v, v))
        predecessor[v].put((cost_v, time_v, u))
      else:
        if cost_v >= P_vertices[v][-1][0] and time_v < P_vertices[v][-1][1]:
          Q.put((cost_v, time_v, v))
          predecessor[v].put((cost_v, time_v, u))

In [6]:
#post-process to extarct the tradeoff curve
P = {}
for p in P_vertices:
  P[p] = []
  for i in range(len(P_vertices[p])):
    P[p].append((P_vertices[p][i][0], P_vertices[p][i][1]))

print("***************************************")
print(f'trade-off curves:\n{P}')
print("***************************************")

***************************************
trade-off curves:
{0: [(0, 0)], 1: [(2, 4), (10, 3)], 3: [(4, 5), (12, 4)], 4: [(3, 9), (4, 2)], 5: [(6, 5), (9, 3)], 2: [(5, 6), (13, 5), (15, 4)], 6: [(7, 9), (10, 6), (13, 4)]}
***************************************


In [7]:
#post-process to extract all the paths found by the algoritm
allPaths = []
def findPathFrom( end, penultimate, totalCForEnd, totalTForEnd, path):
    print("-------------")
    print("From", penultimate)
    print("To", end)
    print("totalCForEnd", totalCForEnd)
    print("totalTForEnd", totalTForEnd)
    
    if penultimate == end and end == 0:
        return True
    print("individual edge",graph[penultimate][end] )
    if totalCForEnd >= graph[penultimate][end][0] and totalTForEnd >= graph[penultimate][end][1]:        
        tempPath = path + [end]
        print("tempPath", tempPath)
        for pct in P_vertices[penultimate]:
            if findPathFrom( penultimate, pct[2], totalCForEnd-graph[penultimate][end][0], totalTForEnd-graph[penultimate][end][1], tempPath):
                print("adding")
                print(tempPath + [0])
                allPaths.append( tempPath + [0])
    else:
        return False

dest = destination
allPaths = []
print("***************************************")
print("destination", dest)
for pct in P_vertices[dest]:
    findPathFrom( dest, pct[2], pct[0], pct[1], [])
print(allPaths)
print("***************************************")

#print fastest cost feasible path
minCost,minTime = float("Inf"),float("Inf")
minPath = None
for aPath in allPaths:
    cost, time = 0, 0
    for i in range( len( aPath)-1, 0, -1):
        cost += graph[aPath[i]][aPath[i-1]][0]
        time += graph[aPath[i]][aPath[i-1]][1]
    print(cost,time)
    if cost <= budget:
        if time < minTime:
            minCost = cost # not really the minCost
            minTime = time
            minPath = aPath
print(f'budget= {budget}')
print(minCost, minTime, minPath)

***************************************
destination 6
-------------
From 2
To 6
totalCForEnd 7
totalTForEnd 9
individual edge (2, 3)
tempPath [6]
-------------
From 1
To 2
totalCForEnd 5
totalTForEnd 6
individual edge (3, 2)
tempPath [6, 2]
-------------
From 0
To 1
totalCForEnd 2
totalTForEnd 4
individual edge (2, 4)
tempPath [6, 2, 1]
-------------
From 0
To 0
totalCForEnd 0
totalTForEnd 0
adding
[6, 2, 1, 0]
-------------
From 4
To 1
totalCForEnd 2
totalTForEnd 4
individual edge (6, 1)
-------------
From 3
To 2
totalCForEnd 5
totalTForEnd 6
individual edge (2, 1)
tempPath [6, 2]
-------------
From 1
To 3
totalCForEnd 3
totalTForEnd 5
individual edge (2, 1)
tempPath [6, 2, 3]
-------------
From 0
To 1
totalCForEnd 1
totalTForEnd 4
individual edge (2, 4)
-------------
From 4
To 1
totalCForEnd 1
totalTForEnd 4
individual edge (6, 1)
-------------
From 1
To 3
totalCForEnd 3
totalTForEnd 5
individual edge (2, 1)
tempPath [6, 2, 3]
-------------
From 0
To 1
totalCForEnd 1
totalTForEnd 4
i